In [1]:
import pygame
import carla
import time
import numpy as np
import cv2

# Connect to the CARLA client
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

traffic_manager = client.get_trafficmanager()


pygame 2.6.1 (SDL 2.28.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
world = client.get_world()
blueprints = world.get_blueprint_library()
vehicle_bp = blueprints.filter('vehicle.dodge.charger_2020')[0]
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[2]

Genenrating the Trajectory

In [3]:
def generate_trajectory(duration=50):
    trajectory = []
    vehicle_test = world.try_spawn_actor(vehicle_bp, start_point)

    spectator = world.get_spectator()

    # Get the transform of the ego vehicle
    transform = vehicle_test.get_transform()

    # Correct way to modify location
    new_location = transform.location + carla.Location(x=-4, z=2.5)

    # Correct way to modify rotation
    new_rotation = carla.Rotation(yaw=transform.rotation.yaw - 15)

    # Apply the new transformation to the spectator
    spectator.set_transform(carla.Transform(new_location, new_rotation))
    spectator.set_transform(transform)

        
    vehicle_test.set_autopilot(True)
    traffic_manager.ignore_lights_percentage(vehicle_test, 100)
    start_time = time.time()

    while time.time() - start_time < duration:
        transform = vehicle_test.get_transform()
        location = transform.location
        trajectory.append([location.x, location.y, location.z])  # Store as list
        time.sleep(0.1)  # Sampling interval

    vehicle_test.set_autopilot(False)
    vehicle_test.destroy()

    return np.array(trajectory)  # Convert list to NumPy array

# Generate trajectory
trajectory_np = generate_trajectory(duration=50)

Visuzlizing it in the CARLA

In [4]:
# Draw the trajectory in CARLA
for waypoint in trajectory_np:
    world.debug.draw_string(carla.Location(x=waypoint[0], y=waypoint[1], z=waypoint[2]), '^',
                            draw_shadow=False, color=carla.Color(r=0, g=255, b=0),
                            life_time=100.0, persistent_lines=True)


Spawing the Car

In [5]:

vehicle = world.try_spawn_actor(vehicle_bp, start_point)

In [6]:
spectator = world.get_spectator()

# Get the transform of the ego vehicle
transform = vehicle.get_transform()

# Correct way to modify location
new_location = transform.location + carla.Location(x=-4, z=2.5)

# Correct way to modify rotation
new_rotation = carla.Rotation(yaw=transform.rotation.yaw - 15)

# Apply the new transformation to the spectator
spectator.set_transform(carla.Transform(new_location, new_rotation))
spectator.set_transform(transform)


Configuring and mounting the Camera

In [7]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
camera_bp.set_attribute('fov', '110')

camera_transform = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Data storage for camera images
camera_data = {'image': np.zeros((360, 640, 3), dtype=np.uint8)}

def camera_callback(image, data_dict):
    """Processes camera images from CARLA and updates data_dict."""
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # BGRA format
    data_dict['image'] = array[:, :, :3]  # Keep only BGR channels

# Attach the camera callback
camera.listen(lambda image: camera_callback(image, camera_data))

Setting up the ppc for Bicycle Model

In [8]:
L = vehicle.bounding_box.extent.x * 2  # Approximate wheelbase
K = 1.0  # Lookahead gain
dt = 0.1  # Control loop time step

def calculate_target(vehicle, trajectory_np, lookahead_distance, start_index):
    vehicle_location = vehicle.get_transform().location
    for i in range(start_index, len(trajectory_np)):
        distance = np.linalg.norm([vehicle_location.x - trajectory_np[i][0],
                                   vehicle_location.y - trajectory_np[i][1]])
        if distance >= lookahead_distance:
            return trajectory_np[i], i
    # Default to the last waypoint if none satisfy the lookahead condition.
    return trajectory_np[-1], len(trajectory_np) - 1


Implementing the Bicycle Model

In [9]:
def pure_pursuit_control(vehicle, target, L, K):

    vehicle_transform = vehicle.get_transform()
    vehicle_location = vehicle_transform.location
    vehicle_yaw = np.deg2rad(vehicle_transform.rotation.yaw)

    velocity = vehicle.get_velocity()
    v = np.linalg.norm([velocity.x, velocity.y, velocity.z])
    
    alpha = np.arctan2(target[1] - vehicle_location.y, target[0] - vehicle_location.x) - vehicle_yaw
    delta = np.arctan(2 * L * np.sin(alpha) / max(K * v, 1.0))
    delta = np.clip(delta, -np.pi / 4, np.pi / 4)
    
    control = carla.VehicleControl()
    control.throttle = 0.5
    control.steer = float(delta / (np.pi / 4))
    vehicle.apply_control(control)


Main Camera and Control Loop

In [10]:
# Initialize Video Writer
video_filename = "output.avi"
frame_width = 640
frame_height = 360
fps = 30  # Frames per second

# Define VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for .avi format
video_writer = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))



waypoint_index = 0  # Initialize lookahead index

try:
    while True:
        # --- Display Camera Feed ---
        frame = camera_data['image']
        cv2.imshow('CARLA Camera View', frame)

        # Save frame to video
        video_writer.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # --- Pure Pursuit Control ---
        velocity = vehicle.get_velocity()
        v = np.linalg.norm([velocity.x, velocity.y, velocity.z])
        lookahead_distance = max(K * v, 1.0)

        # Get the target waypoint using the current index and update the index
        target, waypoint_index = calculate_target(vehicle, trajectory_np, lookahead_distance, waypoint_index)

        # --- Stop Condition: If the vehicle is near the last waypoint ---
        final_waypoint = trajectory_np[-1]
        distance_to_goal = np.linalg.norm([vehicle.get_transform().location.x - final_waypoint[0],
                                           vehicle.get_transform().location.y - final_waypoint[1]])
        
        if distance_to_goal < 2.0:  # Stop when within 2 meters of the last waypoint
            print("Final waypoint reached. Stopping vehicle.")
            control = carla.VehicleControl()
            control.throttle = 0.0
            control.brake = 1.0  # Apply full brake
            control.steer = 0.0
            vehicle.apply_control(control)
            break  # Exit the loop

        pure_pursuit_control(vehicle, target, L, K)
        time.sleep(dt)

except KeyboardInterrupt:
    pass

print(f"Video saved as {video_filename}")


Final waypoint reached. Stopping vehicle.
Video saved as output.avi


In [22]:
cv2.destroyAllWindows()
camera.stop()
camera.destroy()
vehicle.destroy()

True

In [11]:
# OpenCV display loop
while True:
    img = camera_data['image']
    cv2.imshow('CARLA Camera View', img)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cv2.destroyAllWindows()